In [52]:
import numpy as np
import pandas as pd
import csv
import json
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from numpy.linalg import norm
from collections import Counter

In [53]:
# Preprocess
ps = PorterStemmer()

with open("/content/sample_data/stopwords_new.txt") as file:
    stop = file.read()

stopwords = stop.split('\n')

num = list(range(3000))
num = [str(x) for x in num]
stopwords = stopwords + num

def removeStopwords(text):
    for word in text:
        if word in stopwords:
            text.remove(word)
    return text

def tokenize(text):
    token = [] #store token
    sub = "" # store temporary str
    for i in text:
        if i !=" ":
            if i in (",",".","'","?",'"',"@","-","$","!","_","`","(","","*","&","/",")","","''"):
                continue
            sub+=i
            if sub == "\n": #\n 是txt的文字格式，在處理token時就刪除
                sub=""
                continue
        else:
            token.append(sub)
            sub=""
    if sub :
        token.append(sub)
    return token

def preprocessing(text):
#     tokens = [word for word in nltk.word_tokenize(text)]
    tokens = tokenize(text)
    stems = [ps.stem(item) for item in tokens]
    final = []
    return stems

In [54]:
def cosineSim(x, y):
    x_ = x/norm(x)
    y_ = y/norm(y)
    return(np.dot(x_, y_))

In [55]:
# read in tfidf
with open("/content/sample_data/tfidfOverview.csv", 'r') as f:
    reader = csv.reader(f)
    tfidfOverview = np.array([list(map(float, row)) for row in reader])

# read in dictionary
with open("/content/sample_data/dictionary.txt", 'r') as f:
    dictionary = [line.strip() for line in f.readlines()]

# read in idf
with open("/content/sample_data/idf.csv", 'r') as f:
    reader = csv.reader(f)
    idf = np.array([row for row in reader], dtype=object)
    idf = idf.T
    idf = idf.flatten()

### Query testing

In [56]:
dimension = tfidfOverview.shape[1]
query_vector = np.array([0.0 for _ in range(dimension)])

In [57]:
%%time
temp = preprocessing(input())
count = Counter(temp)
for word in count.keys():
    try:
        index = dictionary.index(word)
        query_vector[index] = idf[index]*count[word]
    except:
        continue

war
CPU times: user 75.7 ms, sys: 10.6 ms, total: 86.3 ms
Wall time: 5.08 s


In [58]:
%%time
result = []
for ind in range(1000):
    result.append([cosineSim(query_vector, tfidfOverview[ind]), ind])
sorted_result = sorted(result, key=lambda x: x[0], reverse=True)

CPU times: user 39.6 ms, sys: 0 ns, total: 39.6 ms
Wall time: 53.7 ms


In [59]:
series_titles = []

with open("/content/sample_data/imdb_top_1000.csv", 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        # 將 Series_Title (假設在第一列) 加入列表
        series_titles.append(row[1])

returned_movies = []
for index, item in enumerate(sorted_result):
    if index >= 5:
        break
    if(sorted_result[index][0]>0):
      movie_name = series_titles[item[1]]
      movie_Correlation = sorted_result[index][0]
      returned_movies.append(movie_name)
      print(movie_name)

The Great Escape
Lawrence of Arabia
Hotaru no haka
Empire of the Sun
The Searchers


In [60]:
# for checking movie
df = pd.read_csv('/content/sample_data/imdb_top_1000_preprocessing.csv')
df_movie = df[["Series_Title", "Overview"]]

In [61]:
for i, movie in enumerate(returned_movies):
  print(i, ". ", df_movie.iloc[df.index[df_movie.Series_Title == movie][0]][1])

0 .  Allied prisoners of war plan for several hundred of their number to escape from a German camp during World War II.
1 .  The story of T.E. Lawrence, the English officer who successfully united and led the diverse, often warring, Arab tribes during World War I in order to fight the Turks.
2 .  A young boy and his little sister struggle to survive in Japan during World War II.
3 .  A young English boy struggles to survive under Japanese occupation during World War II.
4 .  An American Civil War veteran embarks on a journey to rescue his niece from the Comanches.


In [62]:
# K = int(input()) # check movie
# print("Title: ", df_movie.iloc[K][0])
# print("Overview: ", df_movie.iloc[K][1])

In [63]:
df_movie.iloc[df.index[df_movie.Series_Title == "Forrest Gump"][0]][1]

'The presidencies of Kennedy and Johnson, the events of Vietnam, Watergate and other historical events unfold through the perspective of an Alabama man with an IQ of 75, whose only desire is to be reunited with his childhood sweetheart.'